In [5]:
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import numpy as np
import os

In [6]:
dataset_dir = 'F:/images'

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.4
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    classes = ["elliptical", "spiral", "uncertain"],
    subset='training'
)

valid_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    classes=["elliptical", "spiral", "uncertain"],
    subset='validation'
)

test_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    classes=["elliptical", "spiral", "uncertain"],
    subset='validation'
)

Found 46356 images belonging to 3 classes.
Found 30902 images belonging to 3 classes.
Found 30902 images belonging to 3 classes.


In [7]:
baseModel = VGG16(
    include_top = False,
    weights = "imagenet",
    input_shape = (224, 224, 3)
)
baseModel.trainable = False

out = baseModel.get_layer('block5_pool').output  # the output of the base model
x = keras.layers.GlobalMaxPooling2D()(out)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(baseModel.input, x)  # the final CNN model

58889256/58889256 [==============================] - 5s 0us/step


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.BinaryCrossentropy(from_logits=True),
    metrics = [keras.metrics.BinaryAccuracy()]
)

history = model.fit(
    train_generator,
    epochs = 10,
    validation_data = valid_generator,
    steps_per_epoch = 46356 // 32,
    validation_steps = 30902 // 32,
    batch_size = 32
)

Epoch 1/10


C:\Users\user\miniconda3\envs\py\lib\site-packages\keras\backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


   4/1448 [..............................] - ETA: 52:03 - loss: 7.2535 - binary_accuracy: 0.5990

KeyboardInterrupt: 